In [ ]:
# main imports for the assignment
import pandas as pd
import numpy as np
import json

# Converting Json to Pandas DataFrame

In [ ]:
# Opening JSON file
f = open('/content/drive/MyDrive/CGAS/archive/train.json',)
 
# returns JSON object as
# a dictionary
data = json.load(f)
 
# Closing file
f.close()

In [ ]:
# creating dataframe from Kaggle recipe data json file
df = pd.DataFrame.from_dict(pd.json_normalize(data), orient='columns')
#df = pd.DataFrame.from_dict('/content/drive/MyDrive/CGAS/archive/train.json')
df

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
...,...,...,...
39769,29109,irish,"[light brown sugar, granulated sugar, butter, ..."
39770,11462,italian,"[KRAFT Zesty Italian Dressing, purple onion, b..."
39771,2238,irish,"[eggs, citrus fruit, raisins, sourdough starte..."
39772,41882,chinese,"[boneless chicken skinless thigh, minced garli..."


In [ ]:
# number of recipes = number of rows in recipe data
len(np.array(df['id'].unique()))

39774

In [ ]:
# for the longest ingredient list in recipe data and the number of recipes 
max_len = 0
count = 0
for i in df['ingredients']:
  count += 1
  length = len(np.array(i))
  if length > max_len:
    max_len = length
  else:
    continue
  
print(max_len)
print(count)

65
39774


In [ ]:
# creating ingredients data by expanding ingredient list in each row of the dataframe
ingredients_data = pd.DataFrame(df['ingredients'].tolist())
#ingredients_data

In [ ]:
# displaying the ingredient data after filling none values with numpy Nan
ingredients_data = ingredients_data.fillna(value=np.nan)
ingredients_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64
0,romaine lettuce,black olives,grape tomatoes,garlic,pepper,purple onion,seasoning,garbanzo beans,feta cheese crumbles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,plain flour,ground pepper,salt,tomatoes,ground black pepper,thyme,eggs,green tomatoes,yellow corn meal,milk,vegetable oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,eggs,pepper,salt,mayonaise,cooking oil,green chilies,grilled chicken breasts,garlic powder,yellow onion,soy sauce,butter,chicken livers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,water,vegetable oil,wheat,salt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,black pepper,shallots,cornflour,cayenne pepper,onions,garlic paste,milk,butter,salt,lemon juice,water,chili powder,passata,oil,ground cumin,boneless chicken skinless thigh,garam masala,double cream,natural yogurt,bay leaf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39769,light brown sugar,granulated sugar,butter,warm water,large eggs,all-purpose flour,whole wheat flour,cooking spray,boiling water,steel-cut oats,dry yeast,salt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39770,KRAFT Zesty Italian Dressing,purple onion,broccoli florets,rotini,pitted black olives,Kraft Grated Parmesan Cheese,red pepper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39771,eggs,citrus fruit,raisins,sourdough starter,flour,hot tea,sugar,ground nutmeg,salt,ground cinnamon,milk,butter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39772,boneless chicken skinless thigh,minced garlic,steamed white rice,baking powder,corn starch,dark soy sauce,kosher salt,peanuts,flour,scallions,Chinese rice vinegar,vodka,fresh ginger,egg whites,broccoli,toasted sesame seeds,sugar,store bought low sodium chicken stock,baking soda,Shaoxing wine,oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
trans = []
for i in range(0, 39774):
    trans.append([str(ingredients_data.values[i,j]) for j in range(0, 65)])

# conveting it into an numpy array
trans = np.array(trans)

# checking the shape of the array
print(trans.shape)
print(trans)

# **For Q1(a)**
## **Apriori Algorithm**

Preprocessing on the Ingredient Data

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
ing_data = te.fit_transform(trans)
ing_data = pd.DataFrame(ing_data, columns = te.columns_)
ing_data
# getting the shape of the data
print(ing_data.shape)

(39774, 6715)


Ingredient Data based on ID from the Kaggle Data

Removing Nan column

In [ ]:
# the data on which apriori algorithm needs to be applied 
ing_data = ing_data.drop(['nan'], axis=1)
ing_data

,( oz.) tomato sauce,( oz.) tomato paste,(10 oz.) frozen chopped spinach,"(10 oz.) frozen chopped spinach, thawed and squeezed dry",(14 oz.) sweetened condensed milk,(14.5 oz.) diced tomatoes,(15 oz.) refried beans,1% low-fat buttermilk,1% low-fat chocolate milk,1% low-fat cottage cheese,1% low-fat milk,"2 1/2 to 3 lb. chicken, cut into serving pieces",2% low fat cheddar chees,2% low-fat cottage cheese,2% lowfat greek yogurt,2% milk shredded mozzarella cheese,2% reduced-fat milk,25% less sodium chicken broth,33% less sodium cooked deli ham,33% less sodium cooked ham,33% less sodium ham,33% less sodium smoked fully cooked ham,40% less sodium taco seasoning,40% less sodium taco seasoning mix,7 Up,"8 ounc ziti pasta, cook and drain",95% lean ground beef,A Taste of Thai Rice Noodles,Accent Seasoning,Adobo All Purpose Seasoning,Alaskan king crab legs,Alexia Waffle Fries,Alfredo sauce,Amarena cherries,Amaretti Cookies,American cheese,Anaheim chile,Angostura bitters,Argo Corn Starch,Asian chili sauce,...,yellow mustard,yellow mustard seeds,yellow onion,yellow peas,yellow peppers,yellow rice,yellow rock sugar,yellow split peas,yellow squash,yellow summer squash,yellow tomato,yellowfin,yellowfin tuna,yellowtail,yellowtail snapper fillets,yoghurt,yoghurt natural low fat,yogurt cheese,yogurt dressing,yogurt low fat,yolk,yoplait,young coconut meat,young leeks,young nettle,yu choy,yuca,yucca,yucca root,yukon gold,yukon gold potatoes,yuzu,yuzu juice,za'atar,zest,zesty italian dressing,zinfandel,ziti,zucchini,zucchini blossoms
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,.

For Apriori on the above preprocessed data

In [ ]:
from mlxtend.frequent_patterns import apriori

Creating itemsets for the above data with minimum possible support to check all itemsets

In [ ]:
# generation of frequent itemsets based on 0.1% minimum support value to display for itemsets of size max length of 6
frequent_itemsets = apriori(ing_data, min_support = 0.001, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.004852,(1% low-fat milk),1
1,0.001785,(2% reduced-fat milk),1
2,0.001207,(Alfredo sauce),1
3,0.001282,(Anaheim chile),1
4,0.001257,(California bay leaves),1
...,...,...,...
26071,0.001031,"(oil, garam masala, ground turmeric, salt, oni...",6
26072,0.001182,"(garam masala, ground turmeric, salt, onions, ...",6
26073,0.001157,"(oil, ground turmeric, salt, onions, tomatoes,...",6
26074,0.001182,"(oil, garam masala, ground turmeric, salt, oni...",6


## **Q1(b)**

In [ ]:
#Printing the itemset of sizes 1,2,3,4 and 5 together with their best support values

size1_freqitem = frequent_itemsets.loc[(frequent_itemsets['length'] == 1) & (frequent_itemsets['support'] == max(np.array(frequent_itemsets.loc[(frequent_itemsets['length'] == 1)]['support'])))]['itemsets']
print("The frequent itemset of size 1 is: ")
print(size1_freqitem)
max_support_size1freq = max(np.array(frequent_itemsets.loc[(frequent_itemsets['length'] == 1)]['support']))
print("The best support for this itemset is: ",max_support_size1freq)
print("\n")
size2_freqitem = frequent_itemsets.loc[(frequent_itemsets['length'] == 2) & (frequent_itemsets['support'] == max(np.array(frequent_itemsets.loc[(frequent_itemsets['length'] == 2)]['support'])))]['itemsets']
print("The frequent itemset of size 2 is: ")
print(size2_freqitem)
max_support_size2freq = max(np.array(frequent_itemsets.loc[(frequent_itemsets['length'] == 2)]['support']))
print("The best support for this itemset is: ",max_support_size2freq)
print("\n")
size3_freqitem = frequent_itemsets.loc[(frequent_itemsets['length'] == 3) & (frequent_itemsets['support'] == max(np.array(frequent_itemsets.loc[(frequent_itemsets['length'] == 3)]['support'])))]['itemsets']
print("The frequent itemset of size 3 is: ")
print(size3_freqitem)
max_support_size3freq = max(np.array(frequent_itemsets.loc[(frequent_itemsets['length'] == 3)]['support']))
print("The best support for this itemset is: ",max_support_size3freq)
print("\n")
size4_freqitem = frequent_itemsets.loc[(frequent_itemsets['length'] == 4) & (frequent_itemsets['support'] == max(np.array(frequent_itemsets.loc[(frequent_itemsets['length'] == 4)]['support'])))]['itemsets']
print("The frequent itemset of size 4 is: ")
print(size4_freqitem)
max_support_size4freq = max(np.array(frequent_itemsets.loc[(frequent_itemsets['length'] == 4)]['support']))
print("The best support for this itemset is: ",max_support_size4freq)
print("\n")
size5_freqitem = frequent_itemsets.loc[(frequent_itemsets['length'] == 5) & (frequent_itemsets['support'] == max(np.array(frequent_itemsets.loc[(frequent_itemsets['length'] == 5)]['support'])))]['itemsets']
print("The frequent itemset of size 5 is: ")
print(size5_freqitem)
max_support_size5freq = max(np.array(frequent_itemsets.loc[(frequent_itemsets['length'] == 5)]['support']))
print("The best support for this itemset is: ",max_support_size5freq)

The frequent itemset of size 1 is: 
912    (salt)
Name: itemsets, dtype: object
The best support for this itemset is:  0.45376376527379697


The frequent itemset of size 2 is: 
9101    (onions, salt)
Name: itemsets, dtype: object
The best support for this itemset is:  0.11042389500678836


The frequent itemset of size 3 is: 
17900    (onions, garlic, salt)
Name: itemsets, dtype: object
The best support for this itemset is:  0.04035299441846432


The frequent itemset of size 4 is: 
24374    (onions, garlic, salt, pepper)
Name: itemsets, dtype: object
The best support for this itemset is:  0.014104691507014632


The frequent itemset of size 5 is: 
25929    (olive oil, pepper, salt, onions, garlic)
Name: itemsets, dtype: object
The best support for this itemset is:  0.004827274098657414


### **Analysis and interpretations:** \newline
1. Salt is the most frequent ingredient item in this recipe dataset.\newline
2. The ingredient item pair of salt and onion is the most frequent used pairs of ingredients in recipes found in this recipe data, which is around 11% support for the frequent itemsets of this recipe data.\newline
3. The frequency of onion, garlic and salt ingredients are highest in frequency when it comes to 3-item frequent itemsets (which is approximately 4% i.e. 1605 recipes out of 39774 recipes).\newline
4. The frequency of onion, garlic, salt and pepper ingredients are highest in frequency when it comes to 4-item frequent itemsets (which is approximately 1% i.e. 561 recipes out of 39774 recipes).\newline
5. The frequency of onion, garlic, salt, pepper and olive oil ingredients are highest in frequency when it comes to 5-item frequent itemsets (which is 192 recipes out of 39774 recipes).\newline
6. As the size of the best support value based frequent-itemset increases, we see that one item adds into the set of the frequent-itemset of size 1 item lesser than the succeeding itemset.\newline
7. As the ingredient complexity increases, their frequency decreases and hence the frequency of their itemsets also decreases proportionally. For instance, 'frozen chopped spinach, thawed and squeezed dry' is a complex ingredient requirement that may be less frequently needed in other types of recipes and hence its support value is lesser than the more simplistic ingredients like 'garlic', 'salt' etc.\newline


# **Generating PDF Report of Jupyter Notebook File**

In [1]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('CGAS_Assignment3_MT20075_Apriori.ipynb')

--2021-11-01 18:01:54--  https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1864 (1.8K) [text/plain]
Saving to: ‘colab_pdf.py’

colab_pdf.py        100%[===================>]   1.82K  --.-KB/s    in 0s      

2021-11-01 18:01:54 (21.5 MB/s) - ‘colab_pdf.py’ saved [1864/1864]





Extracting templates from packages: 100%
[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/CGAS_Assignment3_MT20075_Apriori.ipynb to pdf
[NbConvertApp] Writing 50314 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'File ready to be Downloaded and Saved to Drive'